In [ ]:
import cv2
import numpy as np
import face_recognition
from ultralytics import YOLO
import os

# Load the YOLO model
model = YOLO("K:/fine_tuned/runs/kaggle/working/runs/detect/train2/weights/best.pt")  # Replace with your YOLO model path

# Load known face encodings (initially empty)
reference_face_encoding = None
recognition_threshold = 0.6  # Adjust this threshold as needed

# Function to play a sound
def play_sound(filename):
    os.system(f'start {filename}')  # Works on Windows

def process_frame(frame):
    global reference_face_encoding, recognition_done, tracker, tracking
    
    # Convert the frame to RGB (YOLO model expects RGB images)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Perform YOLO object detection
    results = model(rgb_frame)
    
    face_boxes = []

    # Annotate the frame with bounding boxes
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = box.conf[0]
            class_id = int(box.cls[0])
            class_name = model.names[class_id] if hasattr(model, 'names') else f"Class {class_id}"

            # Store face box coordinates
            face_boxes.append((x1, y1, x2, y2))
            
            if reference_face_encoding is not None:
                detected_face_image = frame[y1:y2, x1:x2]
                if detected_face_image.size > 0:
                    detected_face_image_rgb = cv2.cvtColor(detected_face_image, cv2.COLOR_BGR2RGB)
                    encodings = face_recognition.face_encodings(detected_face_image_rgb)
                    if encodings:
                        detected_face_encoding = encodings[0]
                        face_distance = face_recognition.face_distance([reference_face_encoding], detected_face_encoding)[0]
                        if face_distance < recognition_threshold:
                            # Initialize tracker for the matching face
                            tracker = cv2.TrackerKCF_create()
                            tracker.init(frame, (x1, y1, x2 - x1, y2 - y1))
                            tracking = True
                            recognition_done = True
                            cv2.putText(frame, "Face Matched", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                            play_sound("match_sound.mp3")  # Play a sound on match
                            break

            # Draw rectangle
            color = (0, 255, 0)  # Green for untracked faces
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            # Draw label
            label = f"{class_name} {confidence:.2f}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return frame
